In [29]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from numpy.linalg import norm
# from numpy_ext import rolling_apply as rolling_apply_ext
tqdm.pandas()
import time

In [2]:
raw_df = pd.read_csv('1999-12-01-2023-12-31.csv')
# dates = pd.read_csv('sampled_sector_pairs_200.csv')['']

In [3]:
raw_df['Close'] = raw_df['Adj Close']

In [4]:
# print(raw_df.shape)
# raw_df = raw_df[raw_df.Date.isin(raw_df.Date.unique()[-3059:-3059+1259])]
# print(raw_df.shape)
# print(raw_df.Date.nunique())

In [5]:
raw_df

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry
0,1999-12-01,47.343750,47.625000,46.781250,24.405205,24.405205,2062600.0,MMM,Industrials,Industrial Conglomerates
1,1999-12-02,47.500000,49.218750,47.468750,25.115868,25.115868,2617400.0,MMM,Industrials,Industrial Conglomerates
2,1999-12-03,48.750000,50.406250,48.750000,25.665030,25.665030,3060600.0,MMM,Industrials,Industrial Conglomerates
3,1999-12-06,49.250000,49.531250,48.687500,25.277390,25.277390,1282200.0,MMM,Industrials,Industrial Conglomerates
4,1999-12-07,48.750000,48.750000,46.500000,24.033714,24.033714,2190400.0,MMM,Industrials,Industrial Conglomerates
...,...,...,...,...,...,...,...,...,...,...
2701708,2023-12-22,195.320007,195.910004,192.740005,194.538773,194.538773,1548400.0,ZTS,Health Care,Pharmaceuticals
2701709,2023-12-26,194.880005,196.339996,194.089996,195.057587,195.057587,814600.0,ZTS,Health Care,Pharmaceuticals
2701710,2023-12-27,195.410004,197.009995,194.740005,196.454422,196.454422,766400.0,ZTS,Health Care,Pharmaceuticals
2701711,2023-12-28,197.619995,198.600006,196.529999,196.713837,196.713837,880100.0,ZTS,Health Care,Pharmaceuticals


In [6]:
p1 = raw_df[raw_df.Ticker=='MMM'][['Close','Ticker','Date']].reset_index(drop=True)
p2 = raw_df[raw_df.Ticker=='HD'][['Close','Ticker','Date']].reset_index(drop=True)

In [7]:
df = pd.merge(p1,p2,how='left',on='Date',suffixes=['_P1','_P2'])
df.head()

,Close_P1,Ticker_P1,Date,Close_P2,Ticker_P2
0,24.405205,MMM,1999-12-01,33.109928,HD
1,25.115868,MMM,1999-12-02,33.097225,HD
2,25.665030,MMM,1999-12-03,34.306068,HD
3,25.277390,MMM,1999-12-06,33.937054,HD
4,24.033714,MMM,1999-12-07,33.898861,HD


In [30]:
s = time.time()
df['abs_spread'] = (df['Close_P1'] - df['Close_P2']).abs()
df['abs_spread_mean'] = df.rolling(500).abs_spread.mean()
df['abs_spread_std'] = df.rolling(500).abs_spread.std()

df['abs_spread_mean_l28'] = df.rolling(28).abs_spread.mean()
df['abs_spread_std_l28'] = df.rolling(28).abs_spread.std()
df['spread_normed'] = (df['abs_spread']-df['abs_spread_mean'])/df['abs_spread_std']
df['abs_spread_normed_max'] = df.spread_normed.abs().rolling(500).max()
df['abs_spread_normed_90th'] = df.spread_normed.abs().rolling(500).quantile(0.9)
df['abs_spread_normed_75th'] = df.spread_normed.abs().rolling(500).quantile(0.75)
df['abs_spread_normed_median'] = df.spread_normed.abs().rolling(500).median()
df['abs_spread_normed_l7_avg'] = df.spread_normed.abs().rolling(7).mean()
df['abs_spread_normed_l14_avg'] = df.spread_normed.abs().rolling(14).mean()

e = time.time()
print(e-s)

df.head()

0.01249551773071289


,Close_P1,Ticker_P1,Date,Close_P2,Ticker_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_l28,abs_spread_std_l28,...,entry_signal,exit_signal,stock1_current_Close,stock2_current_Close,stock1_7_days_ago_Close,stock2_7_days_ago_Close,pct_delta_1,pct_delta_2,long_P1,pnl
0,24.405205,MMM,1999-12-01,33.109928,HD,8.704723,NaN,NaN,NaN,NaN,...,0,0,25.115868,33.097225,NaN,NaN,NaN,NaN,1,NaN
1,25.115868,MMM,1999-12-02,33.097225,HD,7.981358,NaN,NaN,NaN,NaN,...,0,0,25.665030,34.306068,NaN,NaN,NaN,NaN,1,NaN
2,25.665030,MMM,1999-12-03,34.306068,HD,8.641039,NaN,NaN,NaN,NaN,...,0,0,25.277390,33.937054,NaN,NaN,NaN,NaN,1,NaN
3,25.277390,MMM,1999-12-06,33.937054,HD,8.659664,NaN,NaN,NaN,NaN,...,0,0,24.033714,33.898861,NaN,NaN,NaN,NaN,1,NaN
4,24.033714,MMM,1999-12-07,33.898861,HD,9.865147,NaN,NaN,NaN,NaN,...,0,0,24.292131,34.325161,NaN,NaN,NaN,NaN,1,NaN


In [9]:
def cos_sim(rs,df):
    rows = df.loc[rs.index]
    vec1_sub1 = rows['Close_P1']
    vec2_sub1 = rows['Close_P2']
    return np.dot(vec1_sub1, vec2_sub1) / (norm(vec1_sub1) * norm(vec2_sub1))

In [10]:
df['cos_sim'] = df['Close_P1'].rolling(500).progress_apply(cos_sim, args=(df,))

0it [00:00, ?it/s]

In [11]:
df['forward_abs_spread'] = df['abs_spread'].shift(-1)

In [12]:
df['entry_thresh'] = df['abs_spread_mean'] + 1.5*df['abs_spread_std']
df['exit_thresh'] = df['abs_spread_mean'] + 0.1*df['abs_spread_std']

In [13]:
df['entry_signal'] = np.where(df['forward_abs_spread']>=df['entry_thresh'],1,0)
df['exit_signal'] = np.where(df['forward_abs_spread']<=df['exit_thresh'],1,0)

In [14]:
df['stock1_current_Close'] = df['Close_P1'].shift(-1)
df['stock2_current_Close'] = df['Close_P2'].shift(-1)
df['stock1_7_days_ago_Close'] = df['Close_P1'].shift(7)
df['stock2_7_days_ago_Close'] = df['Close_P2'].shift(7)
df['pct_delta_1'] = (df['stock1_current_Close']/df['stock1_7_days_ago_Close']) - 1
df['pct_delta_2'] = (df['stock2_current_Close']/df['stock2_7_days_ago_Close']) - 1
df['long_P1'] = np.where(df.pct_delta_1 >= df.pct_delta_2,0,1)

In [15]:
df.shape

(6059, 30)

In [16]:
def compute_pnl(rs,df):
    rows = df.loc[rs.index].iloc[::-1]
    
    long_P1 = rows.long_P1.iloc[-1]

    rows['long_entry'] = np.where(rows.entry_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
    rows['long_exit'] = np.where(rows.exit_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
    rows['short_entry'] = np.where(rows.entry_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)
    rows['short_exit'] = np.where(rows.exit_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)

    rows['exit_signal_sum'] = rows.groupby('entry_signal').exit_signal.cumsum()

    rows['long_entry'] = rows['long_entry'].ffill()
    rows['short_entry'] = rows['short_entry'].ffill()

    rows['long_entry'] = np.where(rows['exit_signal_sum']==1,rows['long_entry'],np.nan)

    rows['long_pnl'] = (rows.long_exit - rows.long_entry)/rows.long_entry
    rows['short_pnl'] = (rows.short_entry - rows.short_exit)/rows.short_entry

    if rows[rows.exit_signal_sum==1].shape[0]>0:
        pnl = rows[rows.exit_signal_sum==1].long_pnl.cumsum().iloc[-1]+rows[rows.exit_signal_sum==1].short_pnl.cumsum().iloc[-1]
    else:
        pnl = np.nan
        
#     if rows.Date.iloc[-1]=='2014-10-06':
#         print(rows)
#         print(rows[rows.exit_signal_sum==1].long_pnl.cumsum().iloc[-1],rows[rows.exit_signal_sum==1].short_pnl.cumsum().iloc[-1])
#         print(pnl)
    return pnl

In [17]:
df['pnl'] = df['Close_P1'].iloc[::-1].rolling(120).progress_apply(compute_pnl, args=(df,))
# .shift(-119)
df['pnl'].dropna().describe(np.arange(0,1,0.1))

0it [00:00, ?it/s]

count    656.000000
mean       0.004899
std        0.112270
min       -0.204320
0%        -0.204320
10%       -0.157504
20%       -0.102653
30%       -0.061336
40%       -0.052679
50%       -0.007146
60%        0.052679
70%        0.088808
80%        0.102653
90%        0.157504
max        0.204320
Name: pnl, dtype: float64

In [ ]:
Everything below is for debugging purpose only

In [18]:
df.dropna().sort_values('pnl',ascending=False)

,Close_P1,Ticker_P1,Date,Close_P2,Ticker_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_l28,abs_spread_std_l28,...,entry_signal,exit_signal,stock1_current_Close,stock2_current_Close,stock1_7_days_ago_Close,stock2_7_days_ago_Close,pct_delta_1,pct_delta_2,long_P1,pnl
5242,138.033447,MMM,2020-10-01,256.683960,HD,118.650513,52.599720,36.638018,114.008730,5.802474,...,1,0,139.398224,258.246552,141.119415,252.699081,-0.012197,0.021953,1,0.20432
5248,147.169601,MMM,2020-10-09,264.358154,HD,117.188553,53.949766,36.924032,113.023712,4.549772,...,1,0,146.604599,265.273438,139.241745,256.767242,0.052878,0.033128,0,0.20432
5233,147.386963,MMM,2020-09-18,254.437286,HD,107.050323,50.716356,36.233435,115.966983,5.588366,...,1,0,140.267502,251.811447,144.040207,256.147766,-0.026192,-0.016929,1,0.20432
5230,144.796478,MMM,2020-09-15,264.043762,HD,119.247284,50.120126,36.074443,116.371553,5.166516,...,1,0,147.352188,260.391602,144.570465,253.919510,0.019241,0.025489,1,0.20432
5234,140.267502,MMM,2020-09-21,251.811447,HD,111.543945,50.916960,36.292060,115.946703,5.603883,...,1,0,141.119415,252.699081,141.823532,252.135132,-0.004965,0.002237,1,0.20432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5228,144.448761,MMM,2020-09-11,255.491257,HD,111.042496,49.708707,35.904331,116.228853,5.158210,...,1,0,146.439438,259.485504,143.257858,262.989746,0.022209,-0.013325,0,-0.20432
5265,143.875031,MMM,2020-11-03,256.452881,HD,112.577850,57.749454,37.227153,115.062364,3.873411,...,1,0,139.224365,261.399445,147.604279,261.658295,-0.056773,-0.000989,1,-0.20432
5266,139.224365,MMM,2020-11-04,261.399445,HD,122.175079,57.992755,37.249927,115.533687,3.908400,...,1,0,141.866974,264.293396,144.440079,255.223160,-0.017814,0.035538,1,-0.20432
5227,141.823532,MMM,2020-09-10,252.135132,HD,110.311600,49.512561,35.836414,116.378986,5.061945,...,1,0,144.448761,255.491257,141.710541,262.161987,0.019323,-0.025445,0,-0.20432


In [26]:
200/5940

0.03367003367003367

In [20]:
n = 3686
3463,3429
df[['Date','stock1_current_Close','stock2_current_Close','Close_P1','Close_P2',\
    'pct_delta_1','pct_delta_2','entry_signal','exit_signal','long_P1']].loc[[3686,3734]]
# .loc[[3686,3734]]
# .loc[n:n+50]
# .iloc[n:n+25]

,Date,stock1_current_Close,stock2_current_Close,Close_P1,Close_P2,pct_delta_1,pct_delta_2,entry_signal,exit_signal,long_P1
3686,2014-07-29,104.974388,65.674637,105.178848,65.048088,-0.007526,0.020979,1,0,1
3734,2014-10-06,101.148216,74.652832,103.102409,75.290604,-0.033689,0.006203,0,1,1


In [21]:
p1_entry = 105.178848
p1_exit = 103.102409

p2_entry = 65.048088
p2_exit = 75.290604

In [22]:
long_entry = p1_entry
long_exit = p1_exit
long = (long_exit - long_entry)/long_entry
long

-0.019741982722609852

In [23]:
short_entry = p2_entry
short_exit = p2_exit
short = (short_entry - short_exit)/short_entry
short

-0.15746067739915728

In [24]:
rows = df[['Date','stock1_current_Close','stock2_current_Close','Close_P1','Close_P2',\
    'pct_delta_1','pct_delta_2','entry_signal','exit_signal','long_P1']].loc[3734-120:3734]
# .loc[3785-120:3785]

long_P1 = rows.long_P1.iloc[-1]

rows['long_entry'] = np.where(rows.entry_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
rows['long_exit'] = np.where(rows.exit_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
rows['short_entry'] = np.where(rows.entry_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)
rows['short_exit'] = np.where(rows.exit_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)

rows['exit_signal_sum'] = rows.groupby('entry_signal').exit_signal.cumsum()

rows['long_entry'] = rows['long_entry'].ffill()
rows['short_entry'] = rows['short_entry'].ffill()

rows['long_entry'] = np.where(rows['exit_signal_sum']==1,rows['long_entry'],np.nan)
rows['short_entry'] = np.where(rows['exit_signal_sum']==1,rows['short_entry'],np.nan)

rows['long_pnl'] = (rows.long_exit - rows.long_entry)/rows.long_entry
rows['short_pnl'] = (rows.short_entry - rows.short_exit)/rows.short_entry

pnl = rows[rows.exit_signal_sum==1].long_pnl.cumsum().iloc[-1]+rows[rows.exit_signal_sum==1].short_pnl.cumsum().iloc[-1]
print(pnl)
rows

-0.15750417393971836


,Date,stock1_current_Close,stock2_current_Close,Close_P1,Close_P2,pct_delta_1,pct_delta_2,entry_signal,exit_signal,long_P1,long_entry,long_exit,short_entry,short_exit,exit_signal_sum,long_pnl,short_pnl
3614,2014-04-15,99.275116,61.156094,97.329826,60.605072,0.006698,-0.027185,1,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
3615,2014-04-16,99.971886,61.563374,99.275116,61.156094,0.024776,-0.000518,1,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
3616,2014-04-17,100.617920,62.258152,99.971886,61.563374,0.028033,0.011023,1,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
3617,2014-04-21,100.813927,63.623741,100.617920,62.258152,0.022453,0.024563,1,0,1,NaN,NaN,NaN,NaN,0,NaN,NaN
3618,2014-04-22,100.160645,62.705353,100.813927,63.623741,0.030007,0.022662,1,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3730,2014-09-30,102.250206,73.482208,104.086906,74.063477,-0.051196,-0.014295,0,0,1,NaN,NaN,NaN,NaN,0,NaN,NaN
3731,2014-10-01,101.875557,74.467140,102.250206,73.482208,-0.048315,0.003809,0,1,1,105.178848,102.250206,65.048088,73.482208,1,-0.027844,-0.12966
3732,2014-10-02,102.977531,75.516647,101.875557,74.467140,-0.029092,0.022407,0,1,1,NaN,101.875557,NaN,74.467140,2,NaN,NaN
3733,2014-10-03,103.102409,75.290604,102.977531,75.516647,-0.031337,0.002580,0,1,1,NaN,102.977531,NaN,75.516647,3,NaN,NaN


In [25]:
rows.entry_signal.value_counts()
rows.exit_signal.value_counts()

exit_signal
0    117
1      4
Name: count, dtype: int64